# Regexp, Interpreters, Compilers


In [75]:
from re import *

In [85]:
def my_search(pattern, text):
    pr1 = compile(pattern)
    return True if pr1.search(text) else False

def my_match(pattern, text):
    pr1 = compile(pattern)
    return True if pr1.match(text) else False

In [86]:
print(my_search('[a-z]', 'MAz'))
my_match('[a-z]', 'aMAZ')

True


True

In [62]:
# formatting strings:
print( '%(language)s has %(#)03d quote types.' % {
    'language': "Python", "#":2  
})

Python has 002 quote types.


In [52]:
#API - appliction programming interface, design of our language:
#lit(s) - literal, ex: lit('a') describes language consisting of only string 'a' {a}
#seq(x, y) - ssequence, ex seq(lit('a'), lit('b')) describes {ab}
#alt(x, y) - alternative, ex: alt(_, _) describes {a, b} - maguage of 2 possibilities a or b
#star(x) - star,  ex: star('a') describes language of(alternatives) {"", a, aa, aaa} - empty or any number of a's
#oneof(c) - oneof, ex: oneof('abc') descr lang {a, b, c} - alternatives (string version of alt)
#eol - end of line, matches eol so {""} - empty string, ex2: seq(lit('a'), eol) matches {a} and nothing else at the end
#dot - dot matcheds any possible character {a, b, c, d, ..}

In [88]:
#
# Your job is to complete this function by filling in the 
# 'dot' and 'oneof' operators to return the correct set of 
# remainders.
#
# dot:   matches any character.
# oneof: matches any of the characters in the string it is 
#        called with. oneof('abc') will match a or b or c.

def matchset(pattern, text):
    "Match pattern at start of text; return a set of remainders of text."
    op, x, y = components(pattern)
    if 'lit' == op:
        return set([text[len(x):]]) if text.startswith(x) else null
    elif 'seq' == op:
        return set(t2 for t1 in matchset(x, text) for t2 in matchset(y, t1))
    elif 'alt' == op:
        return matchset(x, text) | matchset(y, text)
    elif 'dot' == op:
        return set([text[1:]]) if text else null
    elif 'oneof' == op:
        return set([text[1:]]) if text.startswith(x) else null # can be put to startswith if x is a tuple
    elif 'eol' == op:
        return set(['']) if text == '' else null
    elif 'star' == op:
        return (set([text]) |
                set(t2 for t1 in matchset(x, text)
                    for t2 in matchset(pattern, t1) if t1 != text))
    else:
        raise ValueError('unknown pattern: %s' % pattern)
        
null = frozenset()

def components(pattern):
    "Return the op, x, and y arguments; x and y are None if missing."
    x = pattern[1] if len(pattern) > 1 else None
    y = pattern[2] if len(pattern) > 2 else None
    return pattern[0], x, y
   
def test():
    assert matchset(('lit', 'abc'), 'abcdef')            == set(['def'])
    assert matchset(('seq', ('lit', 'hi '),
                     ('lit', 'there ')), 
                   'hi there nice to meet you')          == set(['nice to meet you'])
    assert matchset(('alt', ('lit', 'dog'), 
                    ('lit', 'cat')), 'dog and cat')      == set([' and cat'])
    assert matchset(('dot',), 'am i missing something?') == set(['m i missing something?'])
    assert matchset(('oneof', 'a'), 'aabc123')           == set(['abc123'])
    assert matchset(('eol',),'')                         == set([''])
    assert matchset(('eol',),'not end of line')          == frozenset([])
    assert matchset(('star', ('lit', 'hey')), 'heyhey!') == set(['!', 'heyhey!', 'hey!'])
    
    return 'tests pass'

print(test())

tests pass


In [89]:
# remainding of the API
#---------------
# User Instructions
#
# Fill out the API by completing the entries for alt, 
# star, plus, and eol.


def lit(string):  return ('lit', string)
def seq(x, y):    return ('seq', x, y)
def alt(x, y):    return ('alt', x, y)
def star(x):      return ('star', x)
def plus(x):      return seq(x, star(x))
def opt(x):       return alt(lit(''), x) #opt(x) means that x is optional
def oneof(chars): return ('oneof', tuple(chars))
dot = ('dot',)
eol = ('eol', )

def test():
    assert lit('abc')         == ('lit', 'abc')
    assert seq(('lit', 'a'), 
               ('lit', 'b'))  == ('seq', ('lit', 'a'), ('lit', 'b'))
    assert alt(('lit', 'a'), 
               ('lit', 'b'))  == ('alt', ('lit', 'a'), ('lit', 'b'))
    assert star(('lit', 'a')) == ('star', ('lit', 'a'))
    assert plus(('lit', 'c')) == ('seq', ('lit', 'c'), 
                                  ('star', ('lit', 'c')))
    assert opt(('lit', 'x'))  == ('alt', ('lit', ''), ('lit', 'x'))
    assert oneof('abc')       == ('oneof', ('a', 'b', 'c'))
    return 'tests pass'

print (test())

tests pass


In [90]:
match('[a-z]', 'sAwd')

False

In [91]:
#search without re
def search(pattern, text):
    """Return true if pattern appears anywhere in text
	   Please fill in the match(          , text) below.
	   For example, match(your_code_here, text)"""
    if pattern.startswith('^'):
        return match(pattern[1:], text) # fill this line
    else:
        return match( '.*' + pattern, text) # fill this line

In [92]:
# match without re
def match(pattern, text):
    """
    Return True if pattern appears at the start of text
    
    Please fill in the last line in this program.
    Namely: match(             ,           )

    We'll explain how we came to the code for the condition:
    elif len(pattern) > 1 and pattern[1] in '*?' in the next video lecture
    """

    if pattern == '':
        return True
    elif pattern == '$':
        return (text == '')
    elif len(pattern) > 1 and pattern[1] in '*?':
        p, op, pat = pattern[0], pattern[1], pattern[2:]
        if op == '*':
            return match_star(p, pat, text)
        elif op == '?':
            if match1(p, text) and match(pat, text[1:]):
                return True
            else:
                return match(pat, text)
    else:
        return (match1(pattern[0], text) and
                match(pattern[1:], text[1:]))
def match1(p, text):
    """Returns True if the first character of text matches
    pattern character p"""
    if not text: return False
    return p == '.' or p == text[0]
def match_star(p, pattern, text):
    """Return True if any number of char p, follwed by
    pattern matches text"""
    return match(pattern, text) or (match1(p, text) and match_star(p, pattern, text[1:]))
# recursively against the rest of the text


In [102]:
matchset(lit('a'), 'ab')

{'b'}

In [135]:
# match and search returning strings
#---------------
# User Instructions
#
# Complete the search and match functions. Match should
# match a pattern only at the start of the text. Search
# should match anywhere in the text.

def search(pattern, text):
    "Match pattern anywhere in text; return longest earliest match or None."
    for i in range(len(text)):
        m = match(pattern, text[i:])
        if m is not None:
            return m
        
def match(pattern, text):
    "Match pattern against start of text; return longest match found or None."
    remainders = matchset(pattern, text)
    if remainders:
        shortest = min(remainders, key=len)
        return text[:len(text) - len(shortest)]
    
def components(pattern):
    "Return the op, x, and y arguments; x and y are None if missing."
    x = pattern[1] if len(pattern) > 1 else None
    y = pattern[2] if len(pattern) > 2 else None
    return pattern[0], x, y

def matchset(pattern, text):
    "Match pattern at start of text; return a set of remainders of text."
    op, x, y = components(pattern)
    if 'lit' == op:
        return set([text[len(x):]]) if text.startswith(x) else null
    elif 'seq' == op:
        return set(t2 for t1 in matchset(x, text) for t2 in matchset(y, t1))
    elif 'alt' == op:
        return matchset(x, text) | matchset(y, text)
    elif 'dot' == op:
        return set([text[1:]]) if text else null
    elif 'oneof' == op:
        return set([text[1:]]) if text.startswith(x) else null
    elif 'eol' == op:
        return set(['']) if text == '' else null
    elif 'star' == op:
        return (set([text]) |
                set(t2 for t1 in matchset(x, text)
                    for t2 in matchset(pattern, t1) if t1 != text))
    else:
        raise ValueError('unknown pattern: %s' % pattern)
    
null = frozenset()

#def lit(string):  return ('lit', string)
def lit(s): return lambda text: set([text[len(x):]]) if text.startswith(s) else null
def seq(x, y):    return ('seq', x, y)
#def seq(x, y): return lambda x, y: 
def alt(x, y):    return ('alt', x, y)
def star(x):      return ('star', x)
def plus(x):      return seq(x, star(x))
def opt(x):       return alt(lit(''), x)
def oneof(chars): return ('oneof', tuple(chars))
dot = ('dot',)
eol = ('eol',)

def test():
    assert match(('star', ('lit', 'a')),'aaabcd') == 'aaa'
    assert match(('alt', ('lit', 'b'), ('lit', 'c')), 'ab') == None
    assert match(('alt', ('lit', 'b'), ('lit', 'a')), 'ab') == 'a'
    assert search(('alt', ('lit', 'b'), ('lit', 'c')), 'ab') == 'b'
    return 'tests pass'

print (test())

tests pass


In [112]:
print(search(oneof('abc'), 'Ac'))

c


In [ ]:
#interpreter:
# patterns (regexes):ex:  (a | b)+ defines languages: {a, b, ab, ba, ..,}
# matchset works in that way takes pattern and text and operates on it
# another kind of interpreting language is compiler which takses only pattern
# compiles it returning vompiled object C and executes it on text C(text) (this is faster)

In [129]:
# lests try to change matchset to compiler first, first case lit

def matchset(pattern, text):
    "Match pattern at start of text; return a set of remainders of text."
    op, x, y = components(pattern)
    if 'lit' == op:
        return set([text[len(x):]]) if text.startswith(x) else null
    elif 'seq' == op:
        return set(t2 for t1 in matchset(x, text) for t2 in matchset(y, t1))
    elif 'alt' == op:
        return matchset(x, text) | matchset(y, text)
    elif 'dot' == op:
        return set([text[1:]]) if text else null
    elif 'oneof' == op:
        return set([text[1:]]) if text.startswith(x) else null
    elif 'eol' == op:
        return set(['']) if text == '' else null
    elif 'star' == op:
        return (set([text]) |
                set(t2 for t1 in matchset(x, text)
                    for t2 in matchset(pattern, t1) if t1 != text))
    else:
        raise ValueError('unknown pattern: %s' % pattern)
        
# and literal which retuns tuple, let's make a lit returning function:
# def lit(string):  return ('lit', string) -> 
def lit(s): return lambda text: set([text[len(x):]]) if text.startswith(s) else null


In [130]:
pat1 = lit('a')

In [132]:
pat1('a athing')

{' athing'}

In [134]:
pat2 = plus(pat1) # to do that modifications are needed in seq, plus, ....
pat2

('seq',
 <function __main__.lit.<locals>.<lambda>>,
 ('star', <function __main__.lit.<locals>.<lambda>>))